In [1]:
import numpy as np
import pandas as pd
import joblib
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def clean_and_fill_data(data: pd.DataFrame) -> pd.DataFrame:
    columns_to_drop = ['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature',
                       'FireplaceQu', 'Id', 'GarageYrBlt', '1stFlrSF',
                       'TotRmsAbvGrd', 'GarageArea']
    data.drop(columns=columns_to_drop, axis=1, inplace=True)
    continous_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = data.select_dtypes(exclude=[np.number]).columns.tolist()
    for col in continous_cols:
        data[col] = data[col].fillna(data[col].mean())
    for col in categorical_cols:
        data[col] = data[col].fillna(data[col].mode()[0])
    return data

def scale_continous_features(data, continous_features, model_dir, mode='train') -> np.ndarray:
    scaler_path = os.path.join(model_dir, 'scaler.joblib')
    if mode == 'train':
        scaler = StandardScaler()
        scaler.fit(data[continous_features])
        joblib.dump(scaler, scaler_path)
    else:
        if not os.path.exists(scaler_path):
            raise FileNotFoundError(f"Scaler object not found at {scaler_path}. "
                                    f"Please fit it in train mode.")
        scaler = joblib.load(scaler_path)
        data_scaled = scaler.transform(data[numeric_features])
        return data_scaled

def encode_categorical_features(data, categorical_features, model_dir, mode='train') -> np.ndarray:
    encoder_path = os.path.join(model_dir, 'encoder.joblib')
    if mode == 'train':
        encoder = OneHotEncoder(handle_unknown='ignore')
        encoder.fit(data[categorical_features])
        joblib.dump(encoder, encoder_path)
    else:
        if not os.path.exists(encoder_path):
            raise FileNotFoundError(f"Encoder object not found at {encoder_path}. "
                                    f"Please fit it in train mode.")
        encoder = joblib.load(encoder_path)
        data_encoded = encoder.transform(data[categorical_features]).toarray()
        return data_encoded

def preprocess_features(data, continous_features, categorical_features, model_dir, mode='train') -> np.ndarray:
    data_encoded = encode_categorical_features(data, categorical_features, model_dir, mode)
    data_scaled = scale_continous_features(data, continous_features, model_dir, mode)
    processed_data = np.hstack((data_scaled, data_encoded))
    return processed_data